In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Specify the path to the fastai directory
lib_PATH = '/home/ubuntu/fastai/fastai'   # Update this
!cd {lib_PATH}
# Create sym-link to the fastai library
!ln -s {lib_PATH} ./

ln: failed to create symbolic link './fastai': File exists


In [4]:
# Load required modules
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split

# Load Cleaned Data

In [5]:
train = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_train')
test = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_test')

In [6]:
train.num_plays_by_city.fillna(-1, inplace=True)
train['num_plays_by_city'] = train['num_plays_by_city'].astype(np.int32)
test.num_plays_by_city.fillna(-1, inplace=True)
test['num_plays_by_city'] = test['num_plays_by_city'].astype(np.int32)

In [7]:
df, y, nas = proc_df(train, 'target')    # proc_df deals with na values

# Split into Training + Validation Sets

In [ ]:
# def split_vals(a, n): return a[:n].copy(), a[n:].copy()
# https://www.kaggle.com/kamilkk/i-have-to-say-this
# temporally ordered data - set val. set = test set's length

In [8]:
# len_valid = len(test)
# n_trn = len(df) - len_valid
# X_train, X_valid = split_vals(df, n_trn)
# y_train, y_valid = split_vals(y, n_trn)

X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.25)

X_train.shape, y_train.shape, X_valid.shape

((5533063, 39), (5533063,), (1844355, 39))

# Train the Random Forest

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    auc_trn   = roc_auc_score(y_train, m.predict_proba(X_train)[:, 1])
    auc_valid = roc_auc_score(y_valid, m.predict_proba(X_valid)[:, 1])
    res = [auc_trn, auc_valid]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, auc_valid')
    print(res)

In [ ]:
# set_rf_samples(1_000_000)
reset_rf_samples()

In [ ]:
m = RandomForestClassifier(n_estimators=50, min_samples_leaf=5, n_jobs=-1, verbose=1)
%time m.fit(X_train, y_train)

In [ ]:
print_score(m)

# Retrain Model Using All Data

In [9]:
def print_score_final(m):
    auc_trn = roc_auc_score(y, m.predict_proba(df)[:, 1])
    res = [auc_trn]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn')
    print(res)

In [10]:
# set_rf_samples(1_000_000)
reset_rf_samples()

In [ ]:
m_final = RandomForestClassifier(n_estimators=50, min_samples_leaf=5, n_jobs=-1, verbose=1)
%time m_final.fit(df, y)

In [ ]:
print_score_final(m_final)

# Make Predictions

In [ ]:
# train_cats(test)

In [ ]:
test_df, _, _ = proc_df(test, 'id')

In [ ]:
%time predictions = m_final.predict_proba(test_df)[:, 1]

In [ ]:
output = pd.concat([test['id'], pd.DataFrame(predictions)], axis=1)
output.columns = ['id', 'target']   #Changing column names
output.to_csv('predicted.csv', index=False)